# Zad 1, 2, 3
Wczytaj zbiór danych Credit Card Default, który zawiera informacje o klientach bankowych,
takie jak saldo, limit kredytowy, płeć, wiek itp. Następnie oczyść dane poprzez usuwanie brakujących wartości i kodowanie zmiennych kategorycznych

Zastosuj algorytm K-means do zgrupowania klientów bankowych na podstawie ich zachowań
finansowych. Podziel klientów na kilka klastrów i zbadaj, jak różnią się między sobą

Wykorzystaj metodę Mean Shift do identyfikacji naturalnych grup w danych Credit Card Default. Porównaj wyniki z algorytmem K-means i przeanalizuj, czy różnią się od siebie.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift
import numpy as np 

In [ ]:
# Wczytanie danych
data = pd.read_csv("data/credit.csv")
target = data['default.payment.next.month']
data.drop('default.payment.next.month', axis='columns')
data

In [ ]:
# Usuwanie brakujących wartości
data.dropna(inplace=True)

# Usunięcie duplikatów
data.drop_duplicates(inplace=True)

# Usunięcie niepotrzebnych kolumn
data.drop(['ID'], axis=1, inplace=True)

data['PAY'] = data['PAY_0'] + data['PAY_2'] + data['PAY_3'] + data['PAY_4'] + data['PAY_5'] + data['PAY_6']

data['BILL'] = data['BILL_AMT1'] + data['BILL_AMT2'] + data['BILL_AMT3'] + data['BILL_AMT4'] + data['BILL_AMT6'] + data['BILL_AMT6']

data['PAY_AMT'] = data['PAY_AMT1'] + data['PAY_AMT2'] + data['PAY_AMT3'] + data['PAY_AMT4'] + data['PAY_AMT6']

data.drop(['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6'], axis =1, inplace=True)
data.drop(['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6'], axis=1, inplace=True)
data.drop(['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'], axis=1, inplace=True)

data

In [ ]:
# Inicjalizacja i dopasowanie modelu K-means
k = 2
X = data
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X)
# Przewidywanie przynależności do klastrów dla każdej próbki
labels = kmeans.labels_
# Wyświetlenie wyników
plt.figure(figsize=(10, 6))
plt.scatter(X['AGE'], X['BILL'], c=labels, cmap='viridis', alpha=0.5)
plt.xlabel("Wiek")
plt.ylabel("Bill")
#plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], marker='x', s=200, c='red')
plt.show()

In [ ]:
# Implementacja Mean Shift
ms = MeanShift()
ms.fit(X)
cluster_centers = ms.cluster_centers_
# Zwizualizowanie wyników
plt.figure(figsize=(10, 7))
plt.scatter(X[:, 0], X[:, 1], c=ms.labels_, cmap='viridis')
plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], marker='o', s=300,
 edgecolor='k', facecolor='none')
plt.title('Metoda przesunięcia średniej (Mean Shift)')
plt.xlabel('Cecha 1')
plt.ylabel('Cecha 2')
plt.show()
